In [22]:
import pandas as pd
import sys

def analyze_protein_data(file_path):
    """
    Analyze protein data from an Excel file, checking for duplicates and missing values.
    
    Args:
        file_path (str): Path to the Excel file containing protein data
    """
    try:
        # Read the Excel file
        print(f"\nReading file: {file_path}")
        df = pd.read_excel(file_path, header=1)
        
        # Basic information
        print("\n=== Basic Information ===")
        print(f"Total rows: {len(df)}")
        print(f"Columns: {', '.join(df.columns)}")
        
        # Check for duplicates
        print("\n=== Duplicate Analysis ===")
        duplicates = df[df['Protein'].duplicated(keep=False)]
        if len(duplicates) > 0:
            print(f"Found {len(duplicates)} duplicate proteins:")
            print(duplicates.sort_values('Protein')[['Protein', 'Description']].to_string())
        else:
            print("No duplicate proteins found.")
        
        # Check for missing values
        print("\n=== Missing Value Analysis ===")
        for column in ['Protein', 'Description']:
            missing = df[column].isna().sum()
            print(f"Missing values in {column} column: {missing}")
            if missing > 0:
                print("\nRows with missing values:")
                print(df[df[column].isna()][['Protein', 'Description']].to_string())
        
        
        # Save results to files
        print("\n=== Saving Results ===")
        if len(duplicates) > 0:
            duplicates.to_csv('duplicate_proteins.csv', index=False)
            print("Saved duplicate proteins to 'duplicate_proteins.csv'")
        
        if df['Description'].isna().sum() > 0:
            df[df['Description'].isna()].to_csv('p1_missing_descriptions.csv', index=False)
            print("Saved rows with missing descriptions to 'missing_descriptions.csv'")
        
        # Summary statistics
        print("\n=== Summary Statistics ===")
        print(f"Original data size: {len(df)}")
        #print(f"Cleaned data size: {len(df_cleaned)}")
        print(f"Duplicate entries: {len(duplicates)}")
        print(f"Missing descriptions: {df['Description'].isna().sum()}")
        print(f"Missing proteins: {df['Protein'].isna().sum()}")
        
    except Exception as e:
        print(f"Error analyzing file: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    file_path = '/Users/conny/Desktop/AlphaFold/Summer Project/data/Supplemental_Table_S1.xlsx'
    analyze_protein_data(file_path)


Reading file: /Users/conny/Desktop/AlphaFold/Summer Project/data/Supplemental_Table_S1.xlsx

=== Basic Information ===
Total rows: 803
Columns: Protein, Description, BAMBI fold(hybrid/dsDNA), DPP9 fold(hybrid/dsDNA), Identified by Gromak group?, %disordered_short, %disordered_long, [GS]Y[GS]found?

=== Duplicate Analysis ===
No duplicate proteins found.

=== Missing Value Analysis ===
Missing values in Protein column: 0
Missing values in Description column: 3

Rows with missing values:
       Protein Description
260     HEATR1         NaN
267  HEL-S-103         NaN
298   HLA-DRB1         NaN

=== Saving Results ===
Saved rows with missing descriptions to 'missing_descriptions.csv'

=== Summary Statistics ===
Original data size: 803
Duplicate entries: 0
Missing descriptions: 3
Missing proteins: 0


In [24]:
import requests
import pandas as pd
from tqdm import tqdm

# Read the Excel file
df1 = pd.read_excel('/Users/conny/Desktop/AlphaFold/Summer Project/data/Supplemental_Table_S1.xlsx', header=1)
#df1 = df1.dropna(subset=['Protein', 'Description'])
gene_desc_list = list(zip(df1['Protein'], df1['Description']))

organism = "9606"  # Human
results = []
failed = []
multiple_reviewed = []

# In the main loop where we process each gene
for gene, description in tqdm(gene_desc_list, desc="Querying UniProt"):
    # Modified query to only use gene name, organism, and reviewed status
    query = f'gene_exact:{gene} AND organism_id:{organism} AND reviewed:true'
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": query,
        "fields": "accession,gene_names,protein_name,organism_name",
        "format": "json",
        "size": 500
    }
    try:
        res = requests.get(url, params=params, timeout=30)
        data = res.json()
        results_list = data.get("results", [])
        
        if not results_list:
            failed.append({
                "gene": gene,
                "description": description
            })
        elif len(results_list) > 1:
            # If multiple reviewed entries found, only add to multiple_reviewed
            multiple_reviewed.append({
                "gene": gene,
                "description": description,
                "accessions": [r.get("primaryAccession") for r in results_list],
                "protein_names": [r.get("proteinName", {}).get("value", "") for r in results_list]
            })
        else:
            # Only add to results if there's exactly one reviewed entry
            for result in results_list:
                results.append({
                    "gene": gene,
                    "description": description,
                    "accession": result.get("primaryAccession"),
                    "protein_name": result.get("proteinName", {}).get("value", "")
                })
            
    except Exception as e:
        failed.append({
            "gene": gene,
            "description": description,
            "error": str(e)
        })
        print(f"Error for {gene}: {e}")

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv("paper1_gene_results.csv", index=False)
print("\nSaved main results to paper1_gene_results.csv")

# Save failed queries
if failed:
    failed_df = pd.DataFrame(failed)
    failed_df.to_csv("paper1_failed_queries.csv", index=False)
    print(f"\nFailed queries ({len(failed)}):")
    for item in failed:
        print(f"{item['gene']}: {item['description']}")
    print("\nSaved failed queries to paper1_failed_queries.csv")

# Save multiple reviewed results
if multiple_reviewed:
    multiple_df = pd.DataFrame(multiple_reviewed)
    multiple_df.to_csv("paper1_multiple_reviewed.csv", index=False)
    print(f"\nMultiple reviewed entries ({len(multiple_reviewed)}):")
    for item in multiple_reviewed:
        print(f"\nGene: {item['gene']}")
        print(f"Description: {item['description']}")
        print("Accessions and protein names:")
        for acc, name in zip(item['accessions'], item['protein_names']):
            print(f"  {acc}: {name}")
    print("\nSaved multiple reviewed results to paper1_multiple_reviewed.csv")

Querying UniProt: 100%|██████████| 803/803 [05:30<00:00,  2.43it/s]


Saved main results to paper1_gene_results.csv

Failed queries (49):
CSA2: Protein Red
DKFZp686E1893: Nucleolar protein 9
DKFZp686E2459: RNA-binding protein 10
DKFZp686F18120: Transformer-2 protein homolog beta
DKFZp686L1159: Ras GTPase-activating protein-binding protein 1
EL52: Heat shock protein HSP 90-alpha
hCG_19665: Nuclear pore glycoprotein p62
hCG_1994130: 40S ribosomal protein S15a
hCG_2005638: Spliceosome RNA helicase DDX39B
hCG_2032701: Dolichyl-diphosphooligosaccharide--protein glycosyltransferase subunit STT3A
hCG_27698: Probable ATP-dependent RNA helicase DDX47
hCG_30204: Acetyl-CoA carboxylase 1;Biotin carboxylase
hCG_31253: Far upstream element-binding protein 3
hCG_39985: ATP synthase F(0) complex subunit B1, mitochondrial
HEL113: Vimentin
HEL2: 14-3-3 protein epsilon
HEL70: Moesin
HEL-S-100n: T-complex protein 1 subunit beta
HEL-S-102: Heat shock protein beta-1
HEL-S-103: nan
HEL-S-123m: ATP synthase subunit alpha;ATP synthase subunit alpha, mitochondrial
HEL-S-124m: S

In [28]:
import pandas as pd
import requests
from tqdm import tqdm
import time

# Read the multiple reviewed CSV
multiple_df = pd.read_csv('paper1_multiple_reviewed.csv')
print(f"\nOriginal number of multiple reviewed proteins: {len(multiple_df)}")

# Read existing gene results to append to
existing_results = pd.read_csv('paper1_gene_results.csv')

# Initialize lists for tracking results
new_results = []
still_multiple = []
no_results = []

# Function to clean description for search
def clean_description(desc):
    # Remove common prefixes and clean up the description
    desc = desc.lower()
    # Remove any parenthetical information
    desc = desc.split('(')[0].strip()
    # Remove any semicolons and take first part
    desc = desc.split(';')[0].strip()
    return desc

# Perform refined search for each multiple reviewed entry
for _, row in tqdm(multiple_df.iterrows(), desc="Refining multiple reviewed entries"):
    gene = row['gene']
    description = row['description']
    
    # Clean the description for search
    clean_desc = clean_description(description)
    
    # Create a more specific query using both gene and description
    query = f'gene_exact:{gene} AND organism_id:9606 AND reviewed:true AND protein_name:"{clean_desc}"'
    
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": query,
        "fields": "accession,gene_names,protein_name,organism_name",
        "format": "json",
        "size": 500
    }
    
    try:
        res = requests.get(url, params=params, timeout=30)
        data = res.json()
        results_list = data.get("results", [])
        
        if not results_list:
            # No results found
            no_results.append({
                "gene": gene,
                "description": description,
                "query": query
            })
        elif len(results_list) > 1:
            # Still multiple results
            still_multiple.append({
                "gene": gene,
                "description": description,
                "num_results": len(results_list),
                "accessions": [r.get("primaryAccession") for r in results_list],
                "protein_names": [r.get("proteinName", {}).get("value", "") for r in results_list]
            })
        else:
            # Single result found
            for result in results_list:
                new_results.append({
                    "gene": gene,
                    "description": description,
                    "accession": result.get("primaryAccession"),
                    "protein_name": result.get("proteinName", {}).get("value", "")
                })
            
    except Exception as e:
        print(f"Error for {gene}: {e}")

# Convert new results to DataFrame
new_results_df = pd.DataFrame(new_results)

# Combine with existing results
combined_results = pd.concat([existing_results, new_results_df], ignore_index=True)

# Remove any duplicates based on gene and accession
combined_results = combined_results.drop_duplicates(subset=['gene', 'accession'])

# Save the updated results
combined_results.to_csv("paper1_gene_results_updated.csv", index=False)

# Save still multiple results
if still_multiple:
    still_multiple_df = pd.DataFrame(still_multiple)
    still_multiple_df.to_csv("paper1_still_multiple.csv", index=False)

# Save no results
if no_results:
    no_results_df = pd.DataFrame(no_results)
    no_results_df.to_csv("paper1_no_results.csv", index=False)

# Print detailed summary
print("\n=== Detailed Results Summary ===")
print(f"Original multiple reviewed proteins: {len(multiple_df)}")
print(f"Successfully resolved to single result: {len(new_results)}")
print(f"Still have multiple results: {len(still_multiple)}")
print(f"No results found: {len(no_results)}")

if still_multiple:
    print("\n=== Genes Still Having Multiple Results ===")
    for item in still_multiple:
        print(f"\nGene: {item['gene']}")
        print(f"Description: {item['description']}")
        print(f"Number of results: {item['num_results']}")
        print("Accessions and protein names:")
        for acc, name in zip(item['accessions'], item['protein_names']):
            print(f"  {acc}: {name}")

if no_results:
    print("\n=== Genes With No Results ===")
    for item in no_results:
        print(f"{item['gene']}: {item['description']}")

print(f"\nOriginal results count: {len(existing_results)}")
print(f"New results added: {len(new_results)}")
print(f"Final combined results count: {len(combined_results)}")
print("\nSaved updated results to paper1_gene_results_updated.csv")


Original number of multiple reviewed proteins: 25


Refining multiple reviewed entries: 25it [00:10,  2.46it/s]


=== Detailed Results Summary ===
Original multiple reviewed proteins: 25
Successfully resolved to single result: 21
Still have multiple results: 2
No results found: 2

=== Genes Still Having Multiple Results ===

Gene: RPSA
Description: 40S ribosomal protein SA
Number of results: 2
Accessions and protein names:
  P08865: 
  A0A8I5KQE6: 

Gene: WAS
Description: Wiskott-Aldrich syndrome protein
Number of results: 7
Accessions and protein names:
  P42768: 
  O00401: 
  O43516: 
  Q92558: 
  Q9UPY6: 
  Q9Y6W5: 
  Q9NZQ3: 

=== Genes With No Results ===
CAD: CAD protein;Glutamine-dependent carbamoyl-phosphate synthase;Aspartate carbamoyltransferase;Dihydroorotase
MACF1: Microtubule-actin cross-linking factor 1, isoforms 1/2/3/5

Original results count: 729
New results added: 21
Final combined results count: 750

Saved updated results to paper1_gene_results_updated.csv


In [29]:
import pandas as pd
import requests
from tqdm import tqdm
import time

# Read the failed queries CSV
failed_df = pd.read_csv('paper1_failed_queries.csv')

# Read existing gene results to append to
existing_results = pd.read_csv('paper1_gene_results_updated.csv')

# Initialize lists for tracking results
new_results = []
still_multiple = []
no_results = []

# Perform search for each failed query
for _, row in tqdm(failed_df.iterrows(), desc="Searching failed queries"):
    gene = row['gene']
    description = row['description']
    
    # Simple query with just gene name and taxonomy
    query = f'gene_exact:{gene} AND organism_id:9606'
    
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": query,
        "fields": "accession,gene_names,protein_name,organism_name,reviewed",
        "format": "json",
        "size": 500
    }
    
    try:
        res = requests.get(url, params=params, timeout=30)
        data = res.json()
        results_list = data.get("results", [])
        
        if not results_list:
            # No results found
            no_results.append({
                "gene": gene,
                "description": description
            })
        elif len(results_list) > 1:
            # Multiple results found
            still_multiple.append({
                "gene": gene,
                "description": description,
                "num_results": len(results_list),
                "accessions": [r.get("primaryAccession") for r in results_list],
                "protein_names": [r.get("proteinName", {}).get("value", "") for r in results_list],
                "reviewed_status": [r.get("reviewed", False) for r in results_list]
            })
        else:
            # Single result found
            result = results_list[0]
            new_results.append({
                "gene": gene,
                "description": description,
                "accession": result.get("primaryAccession"),
                "protein_name": result.get("proteinName", {}).get("value", ""),
                "reviewed": result.get("reviewed", False)
            })
            
    except Exception as e:
        print(f"Error for {gene}: {e}")

# Convert new results to DataFrame
new_results_df = pd.DataFrame(new_results)

# Combine with existing results
combined_results = pd.concat([existing_results, new_results_df], ignore_index=True)

# Remove any duplicates based on gene and accession
combined_results = combined_results.drop_duplicates(subset=['gene', 'accession'])

# Save the final results
combined_results.to_csv("paper1_gene_results_final.csv", index=False)

# Print detailed statistics
print(f"\nOriginal results count: {len(existing_results)}")
print(f"New results added: {len(new_results)}")
print(f"Final combined results count: {len(combined_results)}")
print("\nSaved updated results to paper1_gene_results_final.csv")

# Print detailed analysis
print("\nFailed Queries Analysis:")
print(f"Total failed queries processed: {len(failed_df)}")
print(f"Successfully resolved to single result: {len(new_results)}")
print(f"Still have multiple results: {len(still_multiple)}")
print(f"No results found: {len(no_results)}")

if still_multiple:
    print("\n=== Genes Still Having Multiple Results ===")
    for item in still_multiple:
        print(f"\nGene: {item['gene']}")
        print(f"Description: {item['description']}")
        print(f"Number of results: {item['num_results']}")
        print("Accessions and protein names:")
        for acc, name, reviewed in zip(item['accessions'], item['protein_names'], item['reviewed_status']):
            print(f"  {acc}: {name} (Reviewed: {reviewed})")

if no_results:
    print("\n=== Genes With No Results ===")
    for item in no_results:
        print(f"{item['gene']}: {item['description']}")

# Print statistics about reviewed vs unreviewed entries
if 'reviewed' in new_results_df.columns:
    reviewed_count = new_results_df['reviewed'].sum()
    unreviewed_count = len(new_results_df) - reviewed_count
    print(f"\nNew entries statistics:")
    print(f"Reviewed entries: {reviewed_count}")
    print(f"Unreviewed entries: {unreviewed_count}")

Searching failed queries: 49it [00:22,  2.14it/s]


Original results count: 750
New results added: 44
Final combined results count: 794

Saved updated results to paper1_gene_results_final.csv

Failed Queries Analysis:
Total failed queries processed: 49
Successfully resolved to single result: 44
Still have multiple results: 0
No results found: 5

=== Genes With No Results ===
hCG_19665: Nuclear pore glycoprotein p62
hCG_2032701: Dolichyl-diphosphooligosaccharide--protein glycosyltransferase subunit STT3A
hCG_30204: Acetyl-CoA carboxylase 1;Biotin carboxylase
hCG_31253: Far upstream element-binding protein 3
LOC115098: Coiled-coil domain-containing protein 124

New entries statistics:
Reviewed entries: 0
Unreviewed entries: 44


In [30]:
# After the no_results list is populated, add this code:

# Read existing no results file
try:
    existing_no_results = pd.read_csv("paper1_no_results.csv")
    # Combine with new no results
    all_no_results = pd.concat([existing_no_results, pd.DataFrame(no_results)], ignore_index=True)
    # Remove any duplicates based on gene
    all_no_results = all_no_results.drop_duplicates(subset=['gene'])
    # Save back to file
    all_no_results.to_csv("paper1_no_results.csv", index=False)
    print("\nUpdated paper1_no_results.csv with new entries")
except FileNotFoundError:
    # If file doesn't exist, create new one
    pd.DataFrame(no_results).to_csv("paper1_no_results.csv", index=False)
    print("\nCreated new paper1_no_results.csv")


Updated paper1_no_results.csv with new entries


In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import sys
import json
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import random

def fetch_sequence_and_info(protein_id):
    """
    Fetch sequence and protein information from UniProt
    """
    url = f"https://rest.uniprot.org/uniprotkb/{protein_id}"
    params = {"fields": ["sequence", "gene_names", "protein_name"]}
    headers = {"accept": "application/json"}
    
    response = requests.get(url, headers=headers, params=params)
    if response.ok:
        data = response.json()
        sequence = data.get("sequence", {}).get("value", None)
        gene_names = data.get("genes", [{}])[0].get("geneName", {}).get("value", "")
        protein_name = data.get("proteinDescription", {}).get("recommendedName", {}).get("fullName", {}).get("value", "")
        return sequence, gene_names, protein_name
    else:
        print(f"Warning: Failed to fetch {protein_id}", file=sys.stderr)
        return None, None, None

def main():
    # Read the CSV file with accession numbers
    csv_file = "/Users/conny/Desktop/AlphaFold/Summer Project/src/Aggregate/paper1_gene_results_final.csv"
    
    # Read the relevant columns
    df = pd.read_csv(csv_file)
    
    # Extract protein IDs and descriptions
    protein_data = df[['gene', 'accession', 'description']]

    print(f"Processing {len(protein_data)} proteins...")
    
    # Get sequences and create results
    results = []
    for _, row in tqdm(protein_data.iterrows(), desc="Fetching sequences"):
        gene = row['gene']
        accession = row['accession']
        description = row['description']
        
        sequence, gene_names, protein_name = fetch_sequence_and_info(accession)
        if sequence:
            results.append({
                'gene': gene,
                'accession': accession,
                'description': description,
                'protein_name': protein_name,
                'sequence': sequence
            })
    
    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv("paper1_fasta_results.csv", index=False)
    
    # Save sequences to FASTA file
    with open("paper1_fasta_results.fasta", "w") as f:
        for result in results:
            f.write(f">{result['accession']} {result['gene']} {result['protein_name']}\n")
            f.write(f"{result['sequence']}\n")
    
    print("\nResults summary:")
    print(f"- Total proteins processed: {len(protein_data)}")
    print(f"- Successfully retrieved: {len(results)} sequences")
    print("\nOutput files:")
    print("- paper1_fasta_results.csv: Contains all retrieved entries with gene names, accessions, descriptions, and sequences")
    print("- paper1_fasta_results.fasta: Contains protein sequences in FASTA format with gene names in headers")

if __name__ == "__main__":
    main()

Processing 798 proteins...


Fetching sequences: 798it [06:06,  2.18it/s]


Results summary:
- Total proteins processed: 798
- Successfully retrieved: 798 sequences

Output files:
- paper1_fasta_results.csv: Contains all retrieved entries with gene names, accessions, descriptions, and sequences
- paper1_fasta_results.fasta: Contains protein sequences in FASTA format with gene names in headers
